In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [151]:
df = pd.DataFrame()
df['pekerjaan'] = ['karyawan','wiraswasta','honorer','wiraswasta','honorer','honorer','wiraswasta','wiraswasta','honorer',
                   'wiraswasta','karyawan','wiraswasta','honorer','karyawan']
df['penghasilan'] = ['3x Angsuran','3x Angsuran','3x Angsuran','3x Angsuran','3x Angsuran','3x Angsuran','3x Angsuran',
                     '3x Angsuran','3x Angsuran','3x Angsuran','3x Angsuran','3x Angsuran','3x Angsuran','3x Angsuran']
df['sikap'] = ['bermasalah','baik','bermasalah','baik','bermasalah','baik','baik','baik','baik','bermasalah',
               'bermasalah','bermasalah','baik','bermasalah']
df['kepemilikan'] = ['pribadi','pribadi','kontrak','pribadi','pribadi','kontrak','pribadi','kontrak','pribadi','kontrak'
                    ,'kontrak','pribadi','kontrak','kontrak']
df['LayakKredit'] = ['layak','layak','layak','layak','layak','layak','layak',
                     'layak','layak','tidak','tidak','tidak','layak','tidak']

In [152]:
df

,pekerjaan,penghasilan,sikap,kepemilikan,LayakKredit
0,karyawan,3x Angsuran,bermasalah,pribadi,layak
1,wiraswasta,3x Angsuran,baik,pribadi,layak
2,honorer,3x Angsuran,bermasalah,kontrak,layak
3,wiraswasta,3x Angsuran,baik,pribadi,layak
4,honorer,3x Angsuran,bermasalah,pribadi,layak
5,honorer,3x Angsuran,baik,kontrak,layak
6,wiraswasta,3x Angsuran,baik,pribadi,layak
7,wiraswasta,3x Angsuran,baik,kontrak,layak
8,honorer,3x Angsuran,baik,pribadi,layak
9,wiraswasta,3x Angsuran,bermasalah,kontrak,tidak


In [153]:
p_p = len(df.loc[df.LayakKredit == 'layak']) / len(df)
p_n = len(df.loc[df.LayakKredit == 'tidak']) / len(df)

entropy_LayakKredit = -p_n * np.log2(p_n) - p_p * np.log2(p_p)
print(f'H(S) = {entropy_LayakKredit:0.4f}')

H(S) = 0.8631


In [154]:
def f_entropy_LayakKredit(data):
    p_p = len(data.loc[data.LayakKredit == 'layak']) / len(data)
    p_n = len(data.loc[data.LayakKredit == 'tidak']) / len(data)
    return -p_n * np.log2(p_n) - p_p * np.log2(p_p)

In [173]:
IG_LayakKredit_Sikap = entropy_LayakKredit 
overall_eqn = 'Gain(LayakKredit, sikap) = Entropy(LayakKredit)'

for name, Sikap in df.groupby('sikap'):
    num_layak = len(Sikap.loc[Sikap.LayakKredit == 'layak'])
    num_tidak = len(Sikap.loc[Sikap.LayakKredit != 'layak'])
    num_Sikap = len(Sikap)
    print(f'p(LayakKredit=layak|Sikap = {name}) = {num_layak}/{num_Sikap}')
    print(f'p(LayakKredit=tifak|Sikap = {name}) = {num_tidak}/{num_Sikap}')
    print(f'p(pekerjaan = {name}) = {num_Sikap}/{len(df)}')
    print(f'Entropy(LayakKredit|Sikap = {name}) ='\
          f'-{num_layak}/{num_Sikap}.log2({num_layak}/{num_Sikap}) - '\
          f'{num_tidak}/{num_Sikap}.log2({num_tidak}/{num_Sikap})')
    
    entropy_LayakKredit_Sikap = 0
    
    if num_layak != 0:
        entropy_LayakKredit_Sikap -= (num_layak / num_Sikap)\
            *np.log2(num_layak / num_Sikap)
    
    if num_tidak != 0:
        entropy_LayakKredit_Sikap -= (num_tidak / num_Sikap) \
            * np.log2(num_tidak / num_Sikap)
        
    IG_LayakKredit_Sikap -= (num_Sikap / len(df)) * entropy_LayakKredit_Sikap
    print()
    overall_eqn += f'- p(Sikap={name}).'
    overall_eqn += f'Endtropy(Layak Kredit|Sikap={name})'
print(overall_eqn)
print(f'Gain(Layak Kredit, Sikap) = {IG_LayakKredit_Sikap: 0.4f}')

p(LayakKredit=layak|Sikap = baik) = 7/7
p(LayakKredit=tifak|Sikap = baik) = 0/7
p(pekerjaan = baik) = 7/14
Entropy(LayakKredit|Sikap = baik) =-7/7.log2(7/7) - 0/7.log2(0/7)

p(LayakKredit=layak|Sikap = bermasalah) = 3/7
p(LayakKredit=tifak|Sikap = bermasalah) = 4/7
p(pekerjaan = bermasalah) = 7/14
Entropy(LayakKredit|Sikap = bermasalah) =-3/7.log2(3/7) - 4/7.log2(4/7)

Gain(LayakKredit, sikap) = Entropy(LayakKredit)- p(Sikap=baik).Endtropy(Layak Kredit|Sikap=baik)- p(Sikap=bermasalah).Endtropy(Layak Kredit|Sikap=bermasalah)
Gain(Layak Kredit, Sikap) =  0.4257


In [156]:
def IG(data, column, ent_LayakKredit = entropy_LayakKredit):
    IG_LayakKredit = ent_LayakKredit
    for name, temp in data.groupby(column):
        p_p = len(temp.loc[temp.LayakKredit == 'layak']) / len(temp)
        p_n = len(temp.loc[temp.LayakKredit != 'layak']) / len(temp)
        
        entropy_LayakKredit = 0
        
        if p_p != 0:
            entropy_LayakKredit -= (p_p) * np.log2(p_p)
        
        if p_n != 0:
            entropy_LayakKredit -= (p_n) * np.log2(p_n)
        
        IG_LayakKredit -= (len(temp) / len(df)) * entropy_LayakKredit
    return IG_LayakKredit

In [157]:
for col in df.columns[:-1]:
    print(f'Gain(LayakKredit, {col}) = {IG(df, col):0.4f}')

Gain(LayakKredit, pekerjaan) = 0.2728
Gain(LayakKredit, penghasilan) = 0.0000
Gain(LayakKredit, sikap) = 0.3705
Gain(LayakKredit, kepemilikan) = 0.0747


In [181]:
for name, temp in df.groupby('sikap'):
    print('-'*50)
    print(name)
    print('-' *50)
    print(temp)
    print('-' *50)

--------------------------------------------------
baik
--------------------------------------------------
     pekerjaan  penghasilan sikap kepemilikan LayakKredit
1   wiraswasta  3x Angsuran  baik     pribadi       layak
3   wiraswasta  3x Angsuran  baik     pribadi       layak
5      honorer  3x Angsuran  baik     kontrak       layak
6   wiraswasta  3x Angsuran  baik     pribadi       layak
7   wiraswasta  3x Angsuran  baik     kontrak       layak
8      honorer  3x Angsuran  baik     pribadi       layak
12     honorer  3x Angsuran  baik     kontrak       layak
--------------------------------------------------
--------------------------------------------------
bermasalah
--------------------------------------------------
     pekerjaan  penghasilan       sikap kepemilikan LayakKredit
0     karyawan  3x Angsuran  bermasalah     pribadi       layak
2      honorer  3x Angsuran  bermasalah     kontrak       layak
4      honorer  3x Angsuran  bermasalah     pribadi       layak
9   wiras

In [159]:
df_next = df.loc[df.sikap != 'baik']
df_next

,pekerjaan,penghasilan,sikap,kepemilikan,LayakKredit
0,karyawan,3x Angsuran,bermasalah,pribadi,layak
2,honorer,3x Angsuran,bermasalah,kontrak,layak
4,honorer,3x Angsuran,bermasalah,pribadi,layak
9,wiraswasta,3x Angsuran,bermasalah,kontrak,tidak
10,karyawan,3x Angsuran,bermasalah,kontrak,tidak
11,wiraswasta,3x Angsuran,bermasalah,pribadi,tidak
13,karyawan,3x Angsuran,bermasalah,kontrak,tidak


In [160]:
df_karyawan = df_next.loc[df_next.sikap == 'bermasalah']
entropy_LayakKredit = f_entropy_LayakKredit(df_karyawan)
entropy_LayakKredit

0.9852281360342515

In [161]:
for col in df_karyawan.columns[:-1]:
    print(f'(Gain)Layak Kredit, {col} = {IG(df_karyawan, col, entropy_LayakKredit):0.4f}')

(Gain)Layak Kredit, pekerjaan = 0.7885
(Gain)Layak Kredit, penghasilan = 0.4926
(Gain)Layak Kredit, sikap = 0.4926
(Gain)Layak Kredit, kepemilikan = 0.5567


In [186]:
for name, temp in df_karyawan.groupby('pekerjaan'):
    print('-'*60)
    print(name)
    print('-'*60)
    print(temp)
    print('-'*60)

------------------------------------------------------------
honorer
------------------------------------------------------------
  pekerjaan  penghasilan       sikap kepemilikan LayakKredit
2   honorer  3x Angsuran  bermasalah     kontrak       layak
4   honorer  3x Angsuran  bermasalah     pribadi       layak
------------------------------------------------------------
------------------------------------------------------------
karyawan
------------------------------------------------------------
   pekerjaan  penghasilan       sikap kepemilikan LayakKredit
0   karyawan  3x Angsuran  bermasalah     pribadi       layak
10  karyawan  3x Angsuran  bermasalah     kontrak       tidak
13  karyawan  3x Angsuran  bermasalah     kontrak       tidak
------------------------------------------------------------
------------------------------------------------------------
wiraswasta
------------------------------------------------------------
     pekerjaan  penghasilan       sikap kepemilikan L

In [163]:
df_pekerjaan = df_next.loc[df_next.pekerjaan == 'karyawan']
entropy_LayakKredit = f_entropy_LayakKredit(df_pekerjaan)
entropy_LayakKredit

0.9182958340544896

In [164]:
for col in df_pekerjaan.columns[:-1]:
    print(f'(Gain)Layak Kredit, {col} = {IG(df_pekerjaan, col, entropy_LayakKredit):0.4f}')

(Gain)Layak Kredit, pekerjaan = 0.7215
(Gain)Layak Kredit, penghasilan = 0.7215
(Gain)Layak Kredit, sikap = 0.7215
(Gain)Layak Kredit, kepemilikan = 0.9183


In [187]:
for name, temp in df_pekerjaan.groupby('kepemilikan'):
    print('-'*60)
    print(name)
    print('-'*60)
    print(temp)
    print('-'*60)

------------------------------------------------------------
kontrak
------------------------------------------------------------
   pekerjaan  penghasilan       sikap kepemilikan LayakKredit
10  karyawan  3x Angsuran  bermasalah     kontrak       tidak
13  karyawan  3x Angsuran  bermasalah     kontrak       tidak
------------------------------------------------------------
------------------------------------------------------------
pribadi
------------------------------------------------------------
  pekerjaan  penghasilan       sikap kepemilikan LayakKredit
0  karyawan  3x Angsuran  bermasalah     pribadi       layak
------------------------------------------------------------
